# SOUTHHAMPTON v. TOTTENHAM - match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [2]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Arsenal"
away_side = "Everton"

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Southampton"
away_side = "Tottenham"
matchday = "2024-12-15"

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")

Ready to calculate probability for Southampton v. Tottenham


### Calculate Projected Goals

In [3]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.0 2.2857142857142856
1.5714285714285714 1.1428571428571428
1.63 1.38
0.6134969325153374 0.7011393514461 1.1387163561076605 1.6563146997929608

 ================== projected_goals ==================== 

HOME Southampton projected goals: 0.7011393514461
AWAY Tottenham projected goals:  2.602780242531795


### Poisson Distribution of Goal Probabilities

In [4]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Southampton v. Tottenham [[3.67388833e-02 2.57590768e-02 9.03035121e-03 2.11051153e-03
  3.69940671e-04 5.18759925e-05 6.06204995e-06 6.07191682e-07
  5.32157477e-08]
 [9.56232396e-02 6.70452162e-02 2.35040197e-02 5.49319771e-03
  9.62874270e-04 1.35021808e-04 1.57781838e-05 1.58038651e-06
  1.38508897e-07]
 [1.24443139e-01 8.72519821e-02 3.05878991e-02 7.14879324e-03
  1.25307506e-03 1.75716047e-04 2.05335726e-05 2.05669940e-06
  1.80254110e-07]
 [1.07966048e-01 7.56992450e-02 2.65378598e-02 6.20224593e-03
  1.08715967e-03 1.52450086e-04 1.78147923e-05 1.78437885e-06
  1.56387279e-07]
 [7.02529743e-02 4.92571248e-02 1.72680543e-02 4.03577079e-03
  7.07409429e-04 9.91985176e-05 1.15919974e-05 1.16108650e-06
  1.01760430e-07]
 [3.65706107e-02 2.56410943e-02 8.98899010e-03 2.10084490e-03
  3.68246257e-04 5.16383884e-05 6.03428435e-06 6.04410603e-07
  5.29720072e-08]
 [1.58642105e-02 1.11230223e-02 3.89939430e-03 9.11339598e-04
  1.59744014e-04 2.24005628e-05 2.61765268e-06 2.62191329e-07

### Under Market

In [5]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Southampton v. Tottenham
    Goals      Prob  Implied Odds
0    0.5  0.036739     27.219118
1    1.5  0.158121      6.324263
2    2.5  0.358640      2.788312
3    3.5  0.579472      1.725708


In [6]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Southampton v. Tottenham
    Goals      Prob  Implied Odds
0    0.5  0.961763      1.039757
1    1.5  0.840381      1.189936
2    2.5  0.639862      1.562836
3    3.5  0.419030      2.386465


### 1 x 2 Probability

In [7]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
             winner      prob  implied_odds
0  Southampton (1)  0.077420     12.916584
1         Draw (x)  0.141336      7.075338
2    Tottenham (2)  0.779746      1.282468


# Full Results Summary:

In [ ]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


string =  Southampton
Tottenham Hotspur
string =  Tottenham
Tottenham Hotspur
CALCULATED PROBABILITIES FOR Southampton v. Tottenham Hotspur


        ~ ~ moneyline market ~ ~

             winner      prob  implied_odds
0  Southampton (1)  0.077420     12.916584
1         Draw (x)  0.141336      7.075338
2    Tottenham (2)  0.779746      1.282468


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.961763      1.039757
1    1.5  0.840381      1.189936
2    2.5  0.639862      1.562836
3    3.5  0.419030      2.386465


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.036739     27.219118
1    1.5  0.158121      6.324263
2    2.5  0.358640      2.788312
3    3.5  0.579472      1.725708

 ~~ MyBookie Odds for Southampton v. Tottenham Hotspur ~~
      1    x     2  ovr_und_point  over  under
2  4.3  4.6  1.68            3.5  1.74   2.12

 ~~ MyBookie Odds IMPLIED for Southampton v. Tottenham Hotspur ~~
           1         x         

# Conclusion and Pick

What I've learned in playing this game is that Tottenham are always a risky pick. I'm a supporter however, and I'll take a punt on the Coys.

### My Pick:

**Spurs @ 1.68** -- *3/5 unit*